In [1]:
import pandas as pd
import numpy as np

In [2]:
# # very hard to work with formatting so looks messy
# df_ipc=pd.read_excel("Data/Ethiopia - IPC Analysis 2017-2020.xlsx",header=[10,11])
# # df_ipc.columns = [f'{i} {j.replace(".2","")}' if "Unnamed" not in i else f'{j}' for i, j in df_ipc.columns ]
# df_ipcp=df_ipc[['Level 1 Name','Area',"#","Date of Analysis","Area Phase"]]
# df_ipcp=df_ipcp[df_ipcp["Area Phase"].notnull()]
# df_ipcp=df_ipcp.rename(columns={"Level 1 Name":"ADM1_EN","Area":"ADM2_EN","#":"pop_CS","Date of Analysis":"date","Area Phase":"CS"})
# df_ipcp.date=pd.to_datetime(df_ipcp.date,format="%b %Y")
# status="CS"
# for level in [1, 2, 3, 4, 5]:
#     ipc_id = "{}_{}".format(status, level)
#     df_ipcp[ipc_id] = np.where(
#         df_ipcp[status] == level,
#         df_ipcp[f"pop_{status}"],
#         (np.where(df_ipcp[status] == 99, np.nan, 0)),
#     )
# df_ipcp=df_ipcp[["date","ADM1_EN","ADM2_EN","pop_CS"]+[f"CS_{i}" for i in range(1,6)]]
# df_ipcp=df_ipcp.groupby(["date","ADM1_EN"]).agg(lambda x: np.nan if x.isnull().all() else x.sum()).reset_index()

In [3]:
#this is an edited file with solely the admin1 regions and edited column names
df_ipcp=pd.read_csv("Data/GlobalIPC_Ethiopia_Admin1.csv",delimiter=";",thousands=".")

In [4]:
df_ipcp.rename(columns={"Date of Analysis":"date","ADMIN1":"ADM1_EN"},inplace=True)

In [5]:
df_ipcp["date"]=pd.to_datetime(df_ipcp["date"],format="%b/%y")

In [6]:
df_ipcp=df_ipcp.replace('%','', regex=True)

In [7]:
df_ipcp.columns

Index(['ADM1_EN', 'date', 'Country Population', 'pop_CS', 'CS_1', 'CS_1_perc',
       'CS_2', 'CS_2_perc', 'CS_3', 'CS_3_perc', 'CS_4', 'CS_4_perc', 'CS_5',
       'CS_5_perc', 'pop_ML1', 'ML1_1', 'ML1_1_perc', 'ML1_2', 'ML1_2_perc',
       'ML1_3', 'ML1_3_perc', 'ML1_4', 'ML1_4_perc', 'ML1_5', 'ML1_5_perc',
       'pop_ML2', 'ML2_1', 'ML2_1_perc', 'ML2_2', 'ML2_2_perc', 'ML2_3',
       'ML2_3_perc', 'ML2_4', 'ML2_4_perc', 'ML2_5', 'ML2_5_perc'],
      dtype='object')

In [8]:
#convert to ints
df_ipcp.loc[:, ~df_ipcp.columns.isin(["ADM1_EN","date","Analysis Period"])]=df_ipcp.loc[:, ~df_ipcp.columns.isin(["ADM1_EN","date","Analysis Period"])].astype(int)

In [9]:
def get_trigger(row, status, level, perc):
    # range till 6 cause 5 is max level
    cols = [f"{status}_{l}" for l in range(level, 6)]
    if np.isnan(row[f"pop_{status}"]):
        return np.nan
    if row[cols].sum() >= row[f"pop_{status}"] / (100 / perc):
        return 1
    else:
        return 0


def get_trigger_increase(row, level, perc):
    # range till 6 cause 5 is max level
    cols_ml1 = [f"ML1_{l}" for l in range(level, 6)]
    cols_cs = [f"CS_{l}" for l in range(level, 6)]
    if row[["pop_CS", "pop_ML1"]].isnull().values.any():
        return np.nan
    if row[cols_ml1].sum() == 0:
        return 0
    if row[cols_ml1].sum() >= row[cols_cs].sum() * (1 + (perc / 100)):
        return 1
    else:
        return 0

In [10]:
df_ipcp['year'] = df_ipcp['date'].dt.year
df_ipcp['month'] = df_ipcp['date'].dt.month

In [11]:
df_ipcp

,ADM1_EN,date,Country Population,pop_CS,CS_1,CS_1_perc,CS_2,CS_2_perc,CS_3,CS_3_perc,...,ML2_2,ML2_2_perc,ML2_3,ML2_3_perc,ML2_4,ML2_4_perc,ML2_5,ML2_5_perc,year,month
0,Afar,2020-08-01,114964000,1540553,647322,42,516581,34,313370,20,...,583797,37,426361,27,78274,5,0,0,2020,8
1,Amhara,2020-08-01,114964000,6084363,3483150,57,1709456,28,682492,11,...,1188029,19,614339,10,0,0,0,0,2020,8
2,Oromia,2020-08-01,114964000,16937642,7512571,44,5594669,33,3268079,19,...,5565310,32,4326308,25,948789,5,0,0,2020,8
3,SNNP,2020-08-01,114964000,7404334,3582210,48,2393761,32,1112366,15,...,2431700,32,1468883,19,394024,5,0,0,2020,8
4,Sidama,2020-08-01,114964000,3177915,2065645,65,635583,20,476687,15,...,808760,25,647008,20,161752,5,0,0,2020,8
5,Somali,2020-08-01,114964000,5307061,1906949,36,1955920,37,1153513,22,...,1934893,36,1416565,26,542824,10,0,0,2020,8
6,Tigray,2020-08-01,114964000,576351,316993,55,201723,35,57635,10,...,261136,45,58030,10,0,0,0,0,2020,8
7,Afar,2019-09-01,112079000,1513546,620157,40,520180,36,313633,21,...,609136,40,296572,19,44650,2,0,0,2019,9
8,Amhara,2019-09-01,112079000,3457931,1763232,53,871182,26,594367,15,...,1000900,28,675361,19,228197,7,0,0,2019,9
9,Oromiya,2019-09-01,112079000,11497625,3285838,30,4690268,39,2434229,22,...,4439410,38,2862716,24,1301955,10,0,0,2019,9


In [12]:
df_ipcp["trigger_CS_3_20"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",3,20),axis=1)
df_ipcp["trigger_CS_4_2"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,2.5),axis=1)
df_ipcp["trigger_CS_4_20"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,20),axis=1)
df_ipcp["trigger_CS_4_10"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,10),axis=1)
df_ipcp["trigger_ML1_4_10"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",4,10),axis=1)
df_ipcp["trigger_ML1_4_20"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",4,20),axis=1)
df_ipcp["trigger_CS_4_1"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,0.1),axis=1)

In [13]:
def get_trigger(row, status, level, perc):
    # range till 6 cause 5 is max level
    cols = [f"{status}_{l}" for l in range(level, 6)]
    if np.isnan(row[f"pop_{status}"]):
        return np.nan
    if row[cols].sum() >= row[f"pop_{status}"] / (100 / perc):
        return 1
    else:
        return 0


def get_trigger_increase_rel(row, level, perc):
    # range till 6 cause 5 is max level
    cols_ml1 = [f"ML1_{l}" for l in range(level, 6)]
    cols_cs = [f"CS_{l}" for l in range(level, 6)]
    if row[["pop_CS", "pop_ML1"]].isnull().values.any():
        return np.nan
    if row[cols_ml1].sum() == 0:
        return 0
    if row[cols_ml1].sum() >= row[cols_cs].sum() * (1 + (perc / 100)):
        return 1
    else:
        return 0
    
def get_trigger_increase(row, level, perc):
    # range till 6 cause 5 is max level
    cols_ml1 = [f"ML1_{l}" for l in range(level, 6)]
    cols_cs = [f"CS_{l}" for l in range(level, 6)]
    if row[["pop_CS", "pop_ML1"]].isnull().values.any():
        return np.nan
    if row[cols_ml1].sum() == 0:
        return 0
    if (row[cols_ml1].sum() - row[cols_cs].sum())/row["pop_CS"]*100>= perc:
        return 1
    else:
        return 0

In [14]:
#get yes/no for different thresholds
df_ipcp["trigger_CS_3_20"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",3,20),axis=1)
df_ipcp["trigger_CS_3_40"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",3,40),axis=1)
df_ipcp["trigger_CS_4_2"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,2.5),axis=1)
df_ipcp["trigger_CS_4_20"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,20),axis=1)
df_ipcp["trigger_CS_4_10"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,10),axis=1)
df_ipcp["trigger_CS_4_1"]=df_ipcp.apply(lambda x: get_trigger(x,"CS",4,0.1),axis=1)
df_ipcp["trigger_ML1_3_5"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",3,5),axis=1)
df_ipcp["trigger_ML1_4_2"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",4,2.5),axis=1)
df_ipcp["trigger_ML1_4_20"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",4,20),axis=1)
df_ipcp["trigger_ML1_3_20"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",3,20),axis=1)
df_ipcp["trigger_ML1_3_30"]=df_ipcp.apply(lambda x: get_trigger(x,"ML1",3,30),axis=1)
df_ipcp["trigger_ML1_3_5ir"]=df_ipcp.apply(lambda x: get_trigger_increase_rel(x,3,5),axis=1)
df_ipcp["trigger_ML1_3_40ir"]=df_ipcp.apply(lambda x: get_trigger_increase_rel(x,3,40),axis=1)
df_ipcp["trigger_ML1_3_70ir"]=df_ipcp.apply(lambda x: get_trigger_increase_rel(x,3,70),axis=1)
df_ipcp["trigger_ML1_3_5i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,5),axis=1)
df_ipcp["trigger_ML1_3_10i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,10),axis=1)
df_ipcp["trigger_ML1_3_20i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,20),axis=1)
df_ipcp["trigger_ML1_3_30i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,30),axis=1)
df_ipcp["trigger_ML1_3_40i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,40),axis=1)
df_ipcp["trigger_ML1_3_50i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,50),axis=1)
df_ipcp["trigger_ML1_3_70i"]=df_ipcp.apply(lambda x: get_trigger_increase(x,3,70),axis=1)

In [15]:
for c in [f"CS_{i}" for i in range(1,6)]+[f"ML1_{i}" for i in range(1,6)]:
    status=c.split("_")[0]
    df_ipcp[f"perc_{c}"]=df_ipcp[c]/df_ipcp[f"pop_{status}"]*100

In [16]:
df_ipcp

,ADM1_EN,date,Country Population,pop_CS,CS_1,CS_1_perc,CS_2,CS_2_perc,CS_3,CS_3_perc,...,perc_CS_1,perc_CS_2,perc_CS_3,perc_CS_4,perc_CS_5,perc_ML1_1,perc_ML1_2,perc_ML1_3,perc_ML1_4,perc_ML1_5
0,Afar,2020-08-01,114964000,1540553,647322,42,516581,34,313370,20,...,42.018808,33.532180,20.341397,4.107616,0.0,39.435904,37.893471,18.506277,4.164414,0.0
1,Amhara,2020-08-01,114964000,6084363,3483150,57,1709456,28,682492,11,...,57.247571,28.095891,11.217148,3.439374,0.0,85.000007,10.000012,4.999998,0.000000,0.0
2,Oromia,2020-08-01,114964000,16937642,7512571,44,5594669,33,3268079,19,...,44.354291,33.030979,19.294770,3.319954,0.0,48.628587,33.970667,16.454882,0.945864,0.0
3,SNNP,2020-08-01,114964000,7404334,3582210,48,2393761,32,1112366,15,...,48.379908,32.329187,15.023174,4.267744,0.0,48.356733,32.990422,14.577381,4.075478,0.0
4,Sidama,2020-08-01,114964000,3177915,2065645,65,635583,20,476687,15,...,65.000008,20.000000,14.999992,0.000000,0.0,60.000000,25.000008,14.999992,0.000000,0.0
5,Somali,2020-08-01,114964000,5307061,1906949,36,1955920,37,1153513,22,...,35.932298,36.855050,21.735439,5.477175,0.0,38.429651,39.280404,16.860688,5.429239,0.0
6,Tigray,2020-08-01,114964000,576351,316993,55,201723,35,57635,10,...,54.999991,35.000026,9.999983,0.000000,0.0,64.999974,29.999948,5.000078,0.000000,0.0
7,Afar,2019-09-01,112079000,1513546,620157,40,520180,36,313633,21,...,40.973779,34.368298,20.721736,3.936187,0.0,31.238892,37.249149,24.586369,6.925591,0.0
8,Amhara,2019-09-01,112079000,3457931,1763232,53,871182,26,594367,15,...,50.990954,25.193736,17.188515,6.626795,0.0,56.247074,29.101014,11.460321,3.191619,0.0
9,Oromiya,2019-09-01,112079000,11497625,3285838,30,4690268,39,2434229,22,...,28.578406,40.793364,21.171581,9.456649,0.0,29.606694,41.935074,20.677329,7.780902,0.0


In [17]:
dict_an={}

In [18]:
#Analysis: IPC4+ at 20% (current situation)
df_snCS420 = df_ipcp.loc[df_ipcp['trigger_CS_4_20']==1]
display(df_snCS420.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an4"]={"df":df_snCS420,"trig_cols":["CS_4"],"desc":"IPC4+ at 20% (current situation)"}

""


In [19]:
#Analysis: IPC4+ at 10% (current situation)
df_an6 = df_ipcp.loc[df_ipcp['trigger_CS_4_10']==1]
display(df_an6.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an6"]={"df":df_an6,"trig_cols":["CS_4"],"desc":"IPC4+ at 10% (current situation)"}

""


In [20]:
#Analysis: ML14+ at 10% (current situation)
df_an7 = df_ipcp.loc[df_ipcp['trigger_ML1_4_10']==1]
display(df_an7.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an7"]={"df":df_an7,"trig_cols":["ML1_4"],"desc":"ML14+ at 10% (current situation)"}

""


In [21]:
#Analysis: ML14+ at 20% (current situation)
df_an8 = df_ipcp.loc[df_ipcp['trigger_ML1_4_20']==1]
display(df_an8.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an8"]={"df":df_an8,"trig_cols":["ML1_4"],"desc":"ML14+ at 20% (current situation)"}

""


In [22]:
#calculate percentage of population per status and level
for status in ["CS","ML1","ML2"]:
    for i in range(1,6):
        c=f"{status}_{i}"
        df_ipcp[f"perc_{c}"]=df_ipcp[c]/df_ipcp[f"pop_{status}"]*100
    df_ipcp[f"perc_{status}_3p"]=(df_ipcp[[f"{status}_{i}" for i in range(3,6)]].sum(axis=1)/df_ipcp[f"pop_{status}"]*100).round(2)

In [23]:
#Analysis 10: At least 20% of ADMIN1 population currently at IPC3+  AND5% increase in ADMIN1 pop. projected in IPC3+ in 3 months
df_an10 = df_ipcp.loc[(df_ipcp["trigger_CS_3_20"]==1) & (df_ipcp["trigger_ML1_3_5i"]==1)]
# df_an1 = df_ipc.loc[(df_ipc['trigger_CS_3_20']==1)&(df_ipc['trigger_ML1_4_2']==1)]
# display(df_an9[["date","ADM1_EN","perc_ML1_3","perc_ML1_4","perc_CS_3","perc_CS_4","CS_3","CS_4","ML1_3","ML1_4","ML1_i","ML1_i2"]])
display(df_an10.groupby(['year', 'month'], as_index=False)['ADM1_EN','perc_CS_3p','perc_ML1_3p'].agg(lambda x: list(x)))
dict_an["an10"]={"df":df_an10,"trig_cols":["ML1_3","CS_3","CS_4","ML1_4"],"desc":"At least 20% of ADMIN1 population currently at IPC3+  AND5% increase in ADMIN1 pop. projected in IPC3+ in 3 months"}

<ipython-input-23-a9071707f41d>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  display(df_an10.groupby(['year', 'month'], as_index=False)['ADM1_EN','perc_CS_3p','perc_ML1_3p'].agg(lambda x: list(x)))


,year,month,ADM1_EN,perc_CS_3p,perc_ML1_3p
0,2019,9,[Afar],[24.66],[31.51]


In [24]:
#Analysis 10: At least 20% of ADMIN1 population currently at IPC3+  AND5% increase in ADMIN1 pop. projected in IPC3+ in 3 months
df_an10 = df_ipcp.loc[(df_ipcp["trigger_ML1_3_20"]==1) & (df_ipcp["trigger_ML1_3_5i"]==1)]
# df_an1 = df_ipc.loc[(df_ipc['trigger_CS_3_20']==1)&(df_ipc['trigger_ML1_4_2']==1)]
# display(df_an9[["date","ADM1_EN","perc_ML1_3","perc_ML1_4","perc_CS_3","perc_CS_4","CS_3","CS_4","ML1_3","ML1_4","ML1_i","ML1_i2"]])
display(df_an10.groupby(['year', 'month'], as_index=False)['ADM1_EN','perc_CS_3p','perc_ML1_3p'].agg(lambda x: list(x)))
dict_an["an10"]={"df":df_an10,"trig_cols":["ML1_3","CS_3","CS_4","ML1_4"],"desc":"At least 20% of ADMIN1 population currently at IPC3+  AND5% increase in ADMIN1 pop. projected in IPC3+ in 3 months"}

<ipython-input-24-6b7386589f27>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  display(df_an10.groupby(['year', 'month'], as_index=False)['ADM1_EN','perc_CS_3p','perc_ML1_3p'].agg(lambda x: list(x)))


,year,month,ADM1_EN,perc_CS_3p,perc_ML1_3p
0,2019,9,[Afar],[24.66],[31.51]


In [25]:
# #Analysis: IPC3+ at 10% (current situation)
# df_snCS420 = df_ipcp.loc[df_ipcp['trigger_CS_3_20']==1]
# display(df_snCS420.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
# dict_an["an3_20"]={"df":df_snCS420,"trig_cols":["CS_3"],"desc":"IPC3+ at 20% (current situation)"}

In [26]:
#never CS4 value
df_ipcp.perc_CS_4.unique()

array([4.1076159 , 3.43937401, 3.31995445, 4.26774373, 0.        ,
       5.47717465, 3.93618694, 6.62679504, 9.45664866, 3.66720214,
       4.71215587, 2.14139646])

In [27]:
#never CS4 value
df_ipcp.perc_ML1_4.unique()

array([4.16441369, 0.        , 0.94586366, 4.07547796, 5.42923852,
       6.92559063, 3.1916195 , 7.7809019 , 1.47278729, 2.45486623,
       0.99199494])

In [28]:
df_ipcp.columns

Index(['ADM1_EN', 'date', 'Country Population', 'pop_CS', 'CS_1', 'CS_1_perc',
       'CS_2', 'CS_2_perc', 'CS_3', 'CS_3_perc', 'CS_4', 'CS_4_perc', 'CS_5',
       'CS_5_perc', 'pop_ML1', 'ML1_1', 'ML1_1_perc', 'ML1_2', 'ML1_2_perc',
       'ML1_3', 'ML1_3_perc', 'ML1_4', 'ML1_4_perc', 'ML1_5', 'ML1_5_perc',
       'pop_ML2', 'ML2_1', 'ML2_1_perc', 'ML2_2', 'ML2_2_perc', 'ML2_3',
       'ML2_3_perc', 'ML2_4', 'ML2_4_perc', 'ML2_5', 'ML2_5_perc', 'year',
       'month', 'trigger_CS_3_20', 'trigger_CS_4_2', 'trigger_CS_4_20',
       'trigger_CS_4_10', 'trigger_ML1_4_10', 'trigger_ML1_4_20',
       'trigger_CS_4_1', 'trigger_CS_3_40', 'trigger_ML1_3_5',
       'trigger_ML1_4_2', 'trigger_ML1_3_20', 'trigger_ML1_3_30',
       'trigger_ML1_3_5ir', 'trigger_ML1_3_40ir', 'trigger_ML1_3_70ir',
       'trigger_ML1_3_5i', 'trigger_ML1_3_10i', 'trigger_ML1_3_20i',
       'trigger_ML1_3_30i', 'trigger_ML1_3_40i', 'trigger_ML1_3_50i',
       'trigger_ML1_3_70i', 'perc_CS_1', 'perc_CS_2', 'perc_CS_3

### Admin2

In [29]:
df_adt=pd.read_excel("Data/GlobalIPC_simplified_Admin2.xlsx",header=11)

In [30]:
df_adt["date"]=pd.to_datetime(df_adt["date"],format="%b %Y")

In [31]:
df_adt['year'] = df_adt['date'].dt.year
df_adt['month'] = df_adt['date'].dt.month

In [32]:
#only keep the rows related to admin2
df_adt=df_adt[df_adt.ADMIN2.notnull()]

In [33]:
df_adt["ML1_4p_perc"]=df_adt.ML1_4_perc+df_adt.ML1_5_perc

In [34]:
df_adt.value_counts("ML1_4p_perc")

ML1_4p_perc
0.00    59
0.05    55
0.10    11
0.15     2
dtype: int64

In [35]:
#not ever been or forecasted to be IPC 5
df_adt.value_counts("CS")

CS
3.0    99
2.0    28
dtype: int64

In [36]:
df_adtp=df_adt[df_adt.ML1==4]

In [47]:
df_adt[df_adt.ML1_4p_perc>=0.20]

,Country,ADMIN1,ADMIN2,Analysis Name,date,pop_Country,pop_CS,pop_CS_perc,CS,CS_AP,...,ML2_3_perc,ML2_4,ML2_4_perc,ML2_5,ML2_5_perc,ML2_3p,ML2_3p_perc,year,month,ML1_4p_perc


In [37]:
df_adtp

,Country,ADMIN1,ADMIN2,Analysis Name,date,pop_Country,pop_CS,pop_CS_perc,CS,CS_AP,...,ML2_3_perc,ML2_4,ML2_4_perc,ML2_5,ML2_5_perc,ML2_3p,ML2_3p_perc,year,month,ML1_4p_perc


In [39]:
df_adtps=df_adtp[["year","month","ADMIN1","ADMIN2"]]#,"pop_ML1"]]

In [45]:
df_adtps

,year,month,ADMIN1,ADMIN2


In [41]:
df_g=df_adtps.groupby(["year","month","ADMIN1"], as_index=False).agg(lambda x: list(x))

In [44]:
df_g

""


Can you please check for IPC4 >= 20% in ML1 at the ADMIN2 level, then tell us what ADMIN1 regions would have triggered for each year because (a)  at least 1 admin2 region met the threshold and (b) 2 or more admin2 regions met the threshold?

In [ ]:
# print("Analysis a): 1 or more ADMIN2 regions have IPC4 >= 20% in ML1")
# df_g.drop("ADM2_EN",axis=1).set_index(['year', 'month',"ADMIN1"])

In [ ]:
# print("Analysis b): 2 or more ADMIN2 regions have IPC4 >= 20% in ML1")
# df_ant=df_g[df_g["# ADM2 regions ML1 IPC4"]>1]
# df_g[df_g["# ADM2 regions ML1 IPC4"]>1].drop("ADM2_EN",axis=1).set_index(['year', 'month',"ADMIN1"])

### Combine GlobalIPC and Fewsnet

In [ ]:
df_glob=pd.read_csv("Data/GlobalIPC_Ethiopia_Admin1.csv",delimiter=";",thousands=".")

In [ ]:
# df_glob=pd.read_excel("Data/GlobalIPC20192020_Admin1_simplified.xlsx")#,converters={
# #     'percents': lambda value: value * 100})#,delimiter=";",thousands=".")

In [ ]:
df_glob=df_glob.rename(columns={"Date of Analysis":"date"})#,"ADMIN1":"ADM1_EN"},inplace=True)

In [ ]:
df_glob.columns

In [ ]:
df_glob["date"]=pd.to_datetime(df_glob["date"],format="%b/%y")

In [ ]:
df_glob=df_glob.replace('%','', regex=True)


In [ ]:
df_glob.columns

In [ ]:
df_glob.loc[:, ~df_glob.columns.isin(["ADMIN1","date","Analysis Period"])]=df_glob.loc[:, ~df_glob.columns.isin(["ADMIN1","date","Analysis Period"])].astype(int)

In [ ]:
df_glob["Source"]="GlobalIPC"

In [ ]:
df_glob.date=pd.to_datetime(df_glob.date)

In [ ]:
df_glob.loc[df_glob.date.dt.year==2020,"Analysis Period CS"]="Aug 2020 - Sep 2020"
df_glob.loc[df_glob.date.dt.year==2020,"Analysis Period ML1"]="Oct 2020 - Dec 2020"
df_glob.loc[df_glob.date.dt.year==2020,"Analysis Period ML2"]="Jan 2021 - Jun 2021"
df_glob.loc[df_glob.date.dt.year==2019,"Analysis Period CS"]="Jul 2019 - Sep 2019"
df_glob.loc[df_glob.date.dt.year==2019,"Analysis Period ML1"]="Oct 2019 - Jan 2020"
df_glob.loc[df_glob.date.dt.year==2019,"Analysis Period ML2"]="Feb 2020 - Jun 2020"

In [ ]:
df_fews=pd.read_csv("Data/FewsNetPopulation/ethiopia_admin1_fewsnet_population_20090701_20200801.csv",index_col=0)

In [ ]:
df_fews=df_fews[df_fews.date.isin(["2019-10-01","2020-08-01"])].sort_values(["date","ADM1_EN"]).drop("Total",axis=1)#.rename(columns={"adjusted_population":"Country Population"})

In [ ]:
df_fews.date=pd.to_datetime(df_fews.date)

In [ ]:
df_fews.loc[df_fews.date.dt.year==2020,"Analysis Period CS"]="Aug 2020"
df_fews.loc[df_fews.date.dt.year==2020,"Analysis Period ML1"]="Aug 2020 - Sep 2020"
df_fews.loc[df_fews.date.dt.year==2020,"Analysis Period ML2"]="Oct 2020 - Jan 2021"
df_fews.loc[df_fews.date.dt.year==2019,"Analysis Period CS"]="Oct 2019"
df_fews.loc[df_fews.date.dt.year==2019,"Analysis Period ML1"]="Oct 2019 - Jan 2020"
df_fews.loc[df_fews.date.dt.year==2019,"Analysis Period ML2"]="Feb 2020 - May 2020"

In [ ]:
for y in df_fews.date.unique():
    df_fews.loc[df_fews.date==y,"Country Population"]=df_fews.loc[df_fews.date==y,"adjusted_population"].sum()

In [ ]:
df_fews.rename(columns={"ADM1_EN":"ADMIN1"},inplace=True)

In [ ]:
for status in ["CS","ML1","ML2"]:
    for i in range(1,6):
        c=f"{status}_{i}"
        df_fews[f"{c}_perc"]=df_fews[c]/df_fews[f"pop_{status}"]*100
#     df_fews[f'perc_{status}3+']=df_fews[f'{status}3+']/df_fews[f'pop_{status}']*100
#         df_ipcp[f"perc_{c}_miss"]=df_fews[c]/df_fews[f"Country Population"]*100

In [ ]:
df_fews["Source"]="FewsNet"

In [ ]:
df_fews=df_fews.drop("adjusted_population",axis=1)

In [ ]:
df_fews.columns

In [ ]:
df_fews.columns.sort_values()==df_glob.columns.sort_values()

In [ ]:
df_fews.shape

In [ ]:
df_glob.shape

In [ ]:
df_comb=pd.concat([df_glob,df_fews])

In [ ]:
df_comb.head()

In [ ]:
df_comb.date=pd.to_datetime(df_comb.date)

In [ ]:
df_comb["year"]=df_comb.date.dt.year

In [ ]:
for status in ["CS","ML1","ML2"]:
#     print(df_comb[[f'{status}_{i}' for i in range(3,6)]].sum(axis=1))
    df_comb[f'{status}_3+']=df_comb[[f'{status}_{i}' for i in range(3,6)]].sum(axis=1)
    df_comb[f'{status}_3+_perc']=df_comb[f'{status}_3+']/df_comb[f'pop_{status}']*100
    df_comb[f'{status}_2-']=df_comb[[f'{status}_{i}' for i in range(1,3)]].sum(axis=1)
    df_comb[f'{status}_2-_perc']=df_comb[f'{status}_2-']/df_comb[f'pop_{status}']*100

In [ ]:
df_comb=df_comb.sort_values(["year","ADMIN1","Source"])

In [ ]:
df_comb[(df_comb.ADMIN1=="Afar")&(df_comb.Source=="GlobalIPC")]

In [ ]:
perc_cols=[col for col in df_comb.columns if 'perc' in col]
# df_comb[perc_cols]=df_comb[perc_cols]/100

In [ ]:
# df_comb.to_excel("Data/GlobalFews_combined.xlsx")

In [ ]:
ad_gl=df_comb[df_comb.Source=="GlobalIPC"].ADMIN1.unique()

In [ ]:
ad_fn=df_comb[df_comb.Source=="FewsNet"].ADMIN1.unique()

In [ ]:
ad_both=list(set(ad_gl).intersection(set(ad_fn)))

In [ ]:
df_cad=df_comb[df_comb.ADMIN1.isin(ad_both)]

In [ ]:
df_cady=df_cad[df_cad.year==2019]

In [ ]:
df_cadyg=df_cady.groupby("Source",as_index=False).mean()

In [ ]:
df_cadyg

In [ ]:
import plotly.express as px

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_csy=df_comb.groupby(["Source","year"],as_index=False).mean()

In [ ]:
df_csy

In [ ]:
df_csys=df_csy[["Source","year"]+[f"CS_{i}_perc" for i in range(1,6)]]

In [ ]:
df_ysm=df_csy.melt(id_vars=["Source","year"],value_name="percentage",value_vars=['CS_2-_perc','CS_3_perc',"CS_4_perc","CS_5_perc"])#[f"CS_{i}_perc" for i in range(1,6)])
# df_ysm.percentage=df_ysm.percentage*100

In [ ]:
# df_ysm=df_csy.melt(id_vars=["Source","year"],value_name="percentage",value_vars=[f"CS_{i}_perc" for i in range(1,6)]+[f"ML1_{i}_perc" for i in range(1,6)]+[f"ML2_{i}_perc" for i in range(1,6)])
# df_ysm.percentage=df_ysm.percentage*100

In [ ]:
df_ysm

In [ ]:
fig=px.bar(df_ysm,x="variable",y="percentage",color="Source",facet_row="year",barmode="group",height=600,width=800) #,facet_row="year"
fig.update_xaxes(ticktext=["IPC2-","IPC3","IPC4","IPC5"],tickvals=["CS_2-_perc","CS_3_perc","CS_4_perc","CS_5_perc"])
fig.show()

In [ ]:
df_cady[(df_cady.Source=="GlobalIPC")][[f"CS_{i}_perc" for i in range(1,6)]].sum()/len(df_cady[df_cady.Source=="GlobalIPC"])

In [ ]:
# labels = [f"CS_{i}_perc" for i in range(1,6)]
# men_means = df_cadyg[(df_cadyg.Source=="GlobalIPC")][[f"CS_{i}_perc" for i in range(1,6)]].values[0]
# # print(men_means.values[0])
# women_means = df_cadyg[(df_cadyg.Source=="FewsNet")][[f"CS_{i}_perc" for i in range(1,6)]].values[0]

# x = np.arange(len(labels))  # the label locations
# width = 0.35  # the width of the bars

# fig, ax = plt.subplots()
# rects1 = ax.bar(x - width/2, men_means, width, label='GlobalIPC')
# rects2 = ax.bar(x + width/2, women_means, width, label='FewsNet')

# # Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylabel('Scores')
# ax.set_title('Scores by group and gender')
# ax.set_xticks(x)
# ax.set_xticklabels(labels)
# ax.legend()

In [ ]:
df_cady

In [ ]:
df_cs=df_cad[["Source","ADMIN1","year"]+[f"CS_{i}_perc" for i in range(1,6)]]

In [ ]:
df_cs

In [ ]:
df_csm=df_cs.melt(id_vars=["Source","ADMIN1","year"],value_name="percentage")

In [ ]:
df_csm.percentage=df_csm.percentage*100

In [ ]:
df_csm

In [ ]:
df_fews[df_fews.date.dt.year==2020].ML2_4.unique()

In [ ]:
px.bar(df_csm[df_csm.year==2020],x="variable",y="percentage",color="Source",facet_col="ADMIN1",barmode="group",height=300,width=1200) #,facet_row="year"

In [ ]:
import geopandas as gpd
from mapclassify import UserDefined
import mapclassify

In [ ]:
def plot_aff_collist(df_d,cols,shape_path="Data/ET_Admin1_OCHA_2019/eth_admbnda_adm1_csa_bofed_20190827.shp"):
    colp_num=2
    rows=len(cols) // colp_num
    rows+=len(cols) % colp_num
    position = range(1,len(cols) + 1)

    gdf = gpd.read_file(shape_path)
    df_geo=gdf[["ADM1_EN","geometry"]].merge(df_d,left_on="ADM1_EN",right_on="ADMIN1",how="left")
    colors = len(pd.unique(df_geo[cols].values.ravel()))
    cmap = 'Blues'
    figsize = (16, 10)
    scheme = "UserDefined"#"natural_breaks"#'equalinterval'
    fig = plt.figure(1,figsize=(16,6*rows))
    for i,c in enumerate(cols):#range(len(cols)):
  # add every single subplot to the figure with a for loop

        ax = fig.add_subplot(rows,colp_num,position[i])
        df_geo.plot(c, ax=ax,cmap=cmap, figsize=figsize, k = colors,  legend=True,scheme=scheme,classification_kwds=dict(bins=[0,10,20,30,40,50,60,70,80,90,100]))#,missing_kwds={"color":"lightgrey"}) # scheme = scheme,#, figsize=(10, 10),edgecolor='gray',ax=axis,
#                 #    legend=True,
#                 #    legend_kwds={'label': "Cases per 100,000 people",'orientation': "horizontal"},
#                    scheme='Quantiles',k=len(df_geo) scheme="User_Defined", 
        gdf.boundary.plot(linewidth=0.2,ax=ax)

        ax.axis("off")
        plt.title(c)
        if not df_geo[c].isnull().values.all():
            leg = ax.get_legend()

            for lbl in leg.get_texts():
                label_text = lbl.get_text()
            #                 print(label_text)
                upper = label_text.split(",")[-1].rstrip(']')

            #             print("f",upper)
                try:
                    new_text = f'{float(upper):,.2f}'
                except:
                    new_text=upper
                lbl.set_text(new_text)



    plt.show()

In [ ]:
import seaborn as sns

In [ ]:
for i in range(1,6):
    df_comb[f"diffML1_{i}"]=df_comb[f'ML1_{i}_perc']-df_comb[f'CS_{i}_perc']
df_comb[f"diffML3+"]=df_comb[f'ML1_3+_perc']-df_comb[f'CS_3+_perc']

In [ ]:
a=df_comb[(df_comb.date=="2020-08") & (df_comb.Source=="FewsNet")]

In [ ]:
a.value_counts("CS_3_perc").sort_index()

In [ ]:
a.value_counts("diffML1_3").sort_index()

In [ ]:
a.value_counts("diffML3+").sort_index()

In [ ]:
a[["ADMIN1","diffML3+"]]

In [ ]:
df_fewsa=pd.read_csv("Data/FewsNetPopulation/ethiopia_admin1_fewsnet_population_20090701_20200801.csv")

In [ ]:
for status in ["CS","ML1","ML2"]:
    for i in range(1,6):
        c=f"{status}_{i}"
        df_fewsa[f"{c}_perc"]=df_fewsa[c]/df_fewsa[f"pop_{status}"]*100
#     df_fews[f'perc_{status}3+']=df_fews[f'{status}3+']/df_fews[f'pop_{status}']*100
#         df_ipcp[f"perc_{c}_miss"]=df_fews[c]/df_fews[f"Country Population"]*100
    df_fewsa[f'{status}_3+']=df_fewsa[[f'{status}_{i}' for i in range(3,6)]].sum(axis=1)
    df_fewsa[f'{status}_3+_perc']=df_fewsa[f'{status}_3+']/df_fewsa[f'pop_{status}']*100
df_fewsa[f"diffML3+"]=df_fewsa[f'ML1_3+_perc']-df_fewsa[f'CS_3+_perc']
for i in range(1,6):
    df_fewsa[f"diffML1_{i}"]=df_fewsa[f'ML1_{i}_perc']-df_fewsa[f'CS_{i}_perc']

In [ ]:
df_fewsa.value_counts("CS_3_perc").sort_index()

In [ ]:
df_fewsa.value_counts("diffML1_3").sort_index()

In [ ]:
df_fewsa.value_counts("diffML3+").sort_index()

In [ ]:
a=df_fewsa[(df_fewsa.date=="2020-06-01")&(df_fewsa.ADM1_EN=="Harari")]

In [ ]:
a

In [ ]:
a[["date","ADM1_EN","ML1_3_perc","ML1_4_perc","CS_3_perc","CS_4_perc","CS_3","CS_4","ML1_3","ML1_4","diffML3+"]]

In [ ]:
df_fewsa[df_fewsa["diffML3+"]>=40][["date","ADM1_EN","diffML3+"]]

In [ ]:
df_fewsa[df_fewsa["ML1_3_perc"]>=90][["date","ADM1_EN","ML1_3_perc"]].date.unique()

In [ ]:
df_fewsa[df_fewsa["ML1_3_perc"]>=90][["date","ADM1_EN","ML1_3_perc"]]

In [ ]:
len(df_fewsa[df_fewsa.CS_3_perc>=40])

In [ ]:
sns.distplot(df_fewsa.diffML1_3,kde=False)
plt.xlim(20,100)
plt.ylim(0,10)

In [ ]:
plot_aff_collist(df_comb[(df_comb.date=="2020-08") & (df_comb.Source=="FewsNet")],[f"diffML1_{i}" for i in range(1,6)])

In [ ]:
plot_aff_collist(df_comb[(df_comb.date=="2020-08") & (df_comb.Source=="FewsNet")],[f"CS_{i}_perc" for i in range(1,6)])